In [191]:
import spacy
import pandas as pd
import numpy as np
import re


In [192]:
df1=pd.read_csv(r"C:\Users\bmsha\Documents\RoninREbot\data\re_synthetic.csv",encoding="latin1")
df2=pd.read_csv(r"C:\Users\bmsha\Documents\RoninREbot\data\synthetic_RE_common_noise_1000.csv",encoding="latin1")
data=pd.concat([df1,df2],ignore_index=True)
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,labour_charge,total_cost,resolved,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,150,360.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,1331,3928.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,925,3518.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,331,2273.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,150,353.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
data['complaint_type'].value_counts()

complaint_type
Engine              1886
Electrical          1620
Brakes              1459
Drivetrain          1244
Suspension          1184
Body                 991
Wheels               827
noise_issue          528
Exhaust              505
electrical_issue     472
Starting             284
Name: count, dtype: int64

In [194]:
data[data['complaint_type']=='Wheels']

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,labour_charge,total_cost,resolved,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,331,2273.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,RE201600016,Meteor 350,Cruiser,2016,349.0,Cruiser,33037.0,Wheels,Wheel wobbles at high speed since last week,bad bearings,...,474,1823.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,RE202100066,Super Meteor 650,Cruiser,2021,648.0,Cruiser,34274.0,Wheels,Please help  wheel wobbles at high speed,rim damage,...,286,1508.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,RE201600081,Himalayan 411,Historical,2016,411.0,Adventure,44190.0,Wheels,Wheel wobbles at high speed since last week,bad bearings,...,626,3147.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,RE199300088,WD/RE Flying Flea,Historical,1993,125.0,Mini,115600.0,Wheels,Please help  wheel wobbles at high speed,bad bearings,...,501,2631.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9937,RE202309937,Super Meteor 650,Cruiser,2023,648.0,Cruiser,11315.0,Wheels,Wheel wobbles at high speed,bad bearings,...,732,2235.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9948,RE201809948,Super Meteor 650,Cruiser,2018,648.0,Cruiser,28860.0,Wheels,Please help  wheel wobbles at high speed,bad bearings,...,250,1907.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9959,RE202309959,Scram 411,Adventure Crossover,2023,411.0,Adventure,4335.0,Wheels,Wheel wobbles at high speed,bad bearings,...,580,2567.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9967,RE199009967,Classic 500,Historical,1990,500.0,Cruiser,119511.0,Wheels,Wheel wobbles at high speed,rim damage,...,691,2406.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
nlp=spacy.load('en_core_web_lg')

In [196]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,labour_charge,total_cost,resolved,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,150,360.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,1331,3928.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,925,3518.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,331,2273.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,150,353.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
def preprocess(text):
    text=str(text).strip().lower()
    text=re.sub(r"[^a-z\s]"," ",text)
    doc=nlp(text)
    token=[token.lemma_ for token in doc if not token.is_stop and not token.is_punct and len(token.lemma_)>2]
    return " ".join(token)
    
    

In [198]:
data['complints_lemma']=data['complaint'].apply(preprocess)

In [199]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,total_cost,resolved,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost,complints_lemma
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,360.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,engine make knock noise acceleration
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,3928.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate braking week
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,3518.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate brake
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,2273.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wheel wobble high speed
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,353.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,handlebar vibrate high speed


In [200]:
data['initial_advice_lemma']=data['initial_advice'].apply(preprocess)

In [201]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,resolved,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost,complints_lemma,initial_advice_lemma
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,engine make knock noise acceleration,check oil level quality inspect piston ring ti...
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate braking week,check disc drum warp
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate brake,check disc drum warp
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wheel wobble high speed,inspect bearing rim trueness
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,handlebar vibrate high speed,tighten handlebar mount inspect bar end


In [202]:
from sklearn.preprocessing import LabelEncoder

In [203]:
le=LabelEncoder()
data['complaint_label']=le.fit_transform(data['complaint_type'])
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_map)

{'Body': 0, 'Brakes': 1, 'Drivetrain': 2, 'Electrical': 3, 'Engine': 4, 'Exhaust': 5, 'Starting': 6, 'Suspension': 7, 'Wheels': 8, 'electrical_issue': 9, 'noise_issue': 10}


In [204]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,modelname,ec/cc,enginetype,bodytype,kmrunned,estimatedcost,spares_cost,complints_lemma,initial_advice_lemma,complaint_label
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,engine make knock noise acceleration,check oil level quality inspect piston ring ti...,4
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate braking week,check disc drum warp,1
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,brake vibrate brake,check disc drum warp,1
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wheel wobble high speed,inspect bearing rim trueness,8
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,handlebar vibrate high speed,tighten handlebar mount inspect bar end,0


In [205]:
data['Vector']=data['complints_lemma'].apply(lambda text:nlp(text).vector)

In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [207]:
tf=TfidfVectorizer(max_features=5000)
X = tf.fit_transform(data['complints_lemma'])
data['complaint_vec'] = list(X.toarray())

In [208]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,enginetype,bodytype,kmrunned,estimatedcost,spares_cost,complints_lemma,initial_advice_lemma,complaint_label,Vector,complaint_vec
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,NaN,NaN,NaN,NaN,NaN,engine make knock noise acceleration,check oil level quality inspect piston ring ti...,4,"[-0.28434497, 1.3299019, 0.630988, 1.1934401, ...","[0.0, 0.49627939414815236, 0.0, 0.0, 0.0, 0.0,..."
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,NaN,NaN,NaN,NaN,NaN,brake vibrate braking week,check disc drum warp,1,"[1.551, 3.0668252, -1.6211501, 2.8079002, 0.66...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4178324649175..."
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,NaN,NaN,NaN,NaN,NaN,brake vibrate brake,check disc drum warp,1,"[0.32332668, 6.2868667, -2.4179668, 3.4674666,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8823496937838..."
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,NaN,NaN,NaN,NaN,NaN,wheel wobble high speed,inspect bearing rim trueness,8,"[2.2527626, 3.955175, -2.560625, 3.5693998, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,NaN,NaN,NaN,NaN,NaN,handlebar vibrate high speed,tighten handlebar mount inspect bar end,0,"[0.80522, 3.5987248, -2.50495, 3.3589, 1.22955...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [209]:
x_vector=data['complaint_vec']
y=data['complaint_label']

In [210]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,random_state=42,test_size=0.2)

In [211]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(xtrain, ytrain)

MultinomialNB()

In [212]:
from sklearn.metrics import classification_report

In [213]:
ypred=model.predict(xtest)

In [214]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       189
           1       1.00      1.00      1.00       346
           2       0.93      1.00      0.96       246
           3       0.96      1.00      0.98       308
           4       1.00      1.00      1.00       365
           5       1.00      1.00      1.00        91
           6       1.00      1.00      1.00        53
           7       1.00      1.00      1.00       218
           8       1.00      1.00      1.00       171
           9       1.00      0.86      0.92        98
          10       1.00      0.83      0.91       115

    accuracy                           0.98      2200
   macro avg       0.99      0.97      0.98      2200
weighted avg       0.99      0.98      0.98      2200



In [254]:
text=["getting noise from backside of wheel"]
text=preprocess(text)

In [255]:
xtext=tf.transform([text])

In [256]:
pred=model.predict(xtext)

In [257]:
print(pred)

[10]


In [219]:
d=nlp("handle woggling")


In [220]:
d.vector.shape

(300,)

In [221]:
data.head()

,complaint_id,model_name,category,year,engine_cc,body_type,km_run,complaint_type,complaint,initial_diagnosis,...,enginetype,bodytype,kmrunned,estimatedcost,spares_cost,complints_lemma,initial_advice_lemma,complaint_label,Vector,complaint_vec
0,RE201600000,Meteor 350,Cruiser,2016,349.0,Cruiser,38681.0,Engine,Engine makes knocking noise under acceleration,low oil pressure,...,NaN,NaN,NaN,NaN,NaN,engine make knock noise acceleration,check oil level quality inspect piston ring ti...,4,"[-0.28434497, 1.3299019, 0.630988, 1.1934401, ...","[0.0, 0.49627939414815236, 0.0, 0.0, 0.0, 0.0,..."
1,RE202400001,Interceptor 650,Roadster,2024,648.0,Roadster,9012.0,Brakes,Brake vibrates under braking since last week,drum wear,...,NaN,NaN,NaN,NaN,NaN,brake vibrate braking week,check disc drum warp,1,"[1.551, 3.0668252, -1.6211501, 2.8079002, 0.66...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4178324649175..."
2,RE202100002,Interceptor 650,Roadster,2021,648.0,Roadster,30263.0,Brakes,Brake vibrates under braking,drum wear,...,NaN,NaN,NaN,NaN,NaN,brake vibrate brake,check disc drum warp,1,"[0.32332668, 6.2868667, -2.4179668, 3.4674666,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8823496937838..."
3,RE202500003,Shotgun 650,Bobber / Factory Custom,2025,648.0,Bobber,16684.0,Wheels,Wheel wobbles at high speed,bad bearings,...,NaN,NaN,NaN,NaN,NaN,wheel wobble high speed,inspect bearing rim trueness,8,"[2.2527626, 3.955175, -2.560625, 3.5693998, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,RE201700004,Continental GT 650,Café Racer,2017,648.0,Sport,29585.0,Body,Handlebar vibrates at high speed,missing bar weights,...,NaN,NaN,NaN,NaN,NaN,handlebar vibrate high speed,tighten handlebar mount inspect bar end,0,"[0.80522, 3.5987248, -2.50495, 3.3589, 1.22955...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [222]:
xtrain,xtest,ytrain,ytest=train_test_split(data.Vector.values,data.complaint_label	,random_state=42,test_size=0.2)

In [223]:
nb=MultinomialNB()


In [224]:
xtrain2d=np.stack(xtrain)
xtest2d=np.stack(xtest)

In [225]:
from sklearn.preprocessing import MinMaxScaler

In [226]:
scale=MinMaxScaler()
xtrain2d=scale.fit_transform(xtrain2d)

xtest2d=scale.fit_transform(xtest2d)

In [227]:
nb.fit(xtrain2d,ytrain)

MultinomialNB()

In [228]:
ypred=nb.predict(xtest2d)

In [229]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       1.00      0.46      0.63       189
           1       1.00      1.00      1.00       346
           2       1.00      1.00      1.00       246
           3       1.00      0.93      0.96       308
           4       1.00      0.92      0.96       365
           5       1.00      1.00      1.00        91
           6       1.00      1.00      1.00        53
           7       1.00      1.00      1.00       218
           8       0.62      1.00      0.77       171
           9       0.82      1.00      0.90        98
          10       0.79      1.00      0.88       115

    accuracy                           0.93      2200
   macro avg       0.93      0.94      0.92      2200
weighted avg       0.95      0.93      0.93      2200



In [230]:
text="Handlebar woggling"
pre=preprocess(text)

In [231]:
ve=nlp(pre).vector
ve=np.array([ve])

In [232]:
ve2d=scale.transform(ve)

In [233]:
nb.predict(ve2d)

array([3])

In [258]:
import joblib

In [259]:
joblib.dump(tf,r"C:\Users\bmsha\Documents\RoninREbot\models\tfidfvectorizer.pkl")

['C:\\Users\\bmsha\\Documents\\RoninREbot\\models\\tfidfvectorizer.pkl']

In [260]:
joblib.dump(model,r"C:\Users\bmsha\Documents\RoninREbot\models\model.pkl")

['C:\\Users\\bmsha\\Documents\\RoninREbot\\models\\model.pkl']